In [4]:
import numpy as np
import keras 
from keras import backend as K
from keras.layers import Conv2D, Input
from keras.engine.topology import Layer

import tensorflow as tf

Using TensorFlow backend.


In [4]:
# Defining Initializers
prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
prewitt_y = prewitt_x.T 

def init_prewitt_x(shape, dtype=None):
    return K.convert_to_tensor(prewitt_x)

def init_prewitt_y(shape, dtype=None):
    return K.convert_to_tensor(prewitt_y)

In [10]:
# Defining Values
bins = 8         # number of gaps
w = 360/bins     # gap size
centers = np.arange(0, 360, w)+0.5*w

cell_size = 8
block_size = 2
batch_size = 20

In [ ]:
fake_batch = np.random.rand(20, 128, 128, 2)

In [14]:
def hog_layer(conv_stacked_batch):
    #Finding the value of the last axis
    axis = tf.size(tf.shape(conv_stacked_batch))-1 # Ask if considers batch_dim!!!!
    
    #Splitting along axis to x_conv and y_conv
    x_grad, y_grad = tf.split(conv_stacked_batch,2, axis=axis)
    
    # Defining Empty Vote Array for shape (batch_size, img height, img width, bins)
    votes_array = tf.zeros((batch_size, tf.shape(x_grad)[1], tf.shape(x_grad)[2], bins)) 

    def hog_layer_img(votes_array, img_nb,x_grad, y_grad):
        #Calculating magnitudes
        mags = tf.sqrt(tf.pow(x_grad[img_nb, :, :], 2)+tf.pow(y_grad[img_nb, :, :], 2))

        #Calculating angles from 0 to 360 degrees
        angles = 180*(tf.atan2(y_grad[img_nb, :, :], x_grad[img_nb, :, :]) + np.pi)/np.pi 

        # Defining the function that generates all votes for each bin number
        def create_vote_array(votes_array, angles, mags, bin_nb):
            # The Else function is for all bins expect for the last bin
            # Voting for angles that lie in the last bin act differently (see bellow)

            # For angles in bin number (bin_nb=j), where bin j has boundries center[i] to center[i+1]
            # where bin j has boundries center[i] to center[i+1]
            # unless bin j is the last bin, then the boutreis are from the last center to the first center
            # If the bins=8, the center would be:
            # [ 22.5,   67.5,  112.5,  157.5,  202.5,  247.5,  292.5,  337.5]

            # Read --else-- comments first for explanation 
            if bin_nb == bins-1:
                # Since it is hard to express in python that an angle 0 = angle 360, we do this instead
                # Adding 360 to where the center and the angles are not both on the same side of the 0/360 boundary
                # This satisfies vote_j + vote_j+1 = mag
                votes_top_j = tf.where(centers[-1] < angles, tf.multiply(mags, (centers[0]-angle_array+360)/w), tf.zeros(tf.shape(angles)))
                votes_top_j1 = tf.where(centers[-1] < angles, tf.multiply(mags, (angle_array-centers[bin_nb])/w), tf.zeros(tf.shape(angles)))

                votes_bot_j = tf.where(centers[0] > angles, tf.multiply(mags, (centers[0]-angle_array)/w), tf.zeros(tf.shape(angles)))
                votes_bot_j1 = tf.where(centers[0] > angles, tf.multiply(mags, (360+angle_array-centers[bin_nb])/w), tf.zeros(tf.shape(angles)))


                votes_j = tf.add(votes_top_j, votes_bot_j)
                votes_j1 = tf.add(votes_top_j1, votes_bot_j1)

                votes_array[:, :, bin_nb] += votes_j
                votes_array[:, :, 0] += votes_j1

            # finds all angles in bin j
            # for those not in bin j, element = 0
            else:
                # Calculates votes for bin j for angles that are in bin j
                votes_j = tf.where((centers[bin_nb]) < angles < centers[bin_nb+1], tf.multiply(mags, (centers[bin_nb+1]-angles)/w), tf.zeros(tf.shape(angles)))

                #Calculates votes for bin j+1 for angles that are in bin j
                votes_j1 = tf.where((centers[bin_nb]) < angles < centers[bin_nb+1], tf.multiply(mags, (angle_array-centers[bin_nb])/w), tf.zeros(tf.shape(angles)))

                #Adds votes the bin jth and j+1th layer on axis=2 of voting array                            
                votes_array[img_nb, :, :, bin_nb] = tf.add(votes_array[img_nb, :, :, bin_nb], votes_j)
                votes_array[img_nb, :, :, bin_nb+1] = tf.add(votes_array[img_nb, :, :, bin_nb+1], votes_j1)

            return votes_array
            #Bin Level
        
        # Interating through each bin to generate votes                           
        for i in range(bins):
            votes_array = create_vote_array(votes_array, angles, mags, i)
        
        return votes_array
        # Image Level                            
    
    # Performing hog_layer_img function on each imgage in the batch
    for img_nb in range(batch_size):
        votes_array = hog_layer_img(votes_array, img_nb, x_grad, y_grad)
                                
    # For each cell, calculating the average vote at each bin level (or depth)
    # We multiply each element by the number of pixels in the cell, giving the sum of the votes for each bin level   
    # Result is array that keeps the spatial infomation of the cells
    # and the last dimetion (with depth bins) is the cell histogram                            
    cell_array = (cell_size**2)*tf.nn.avg_pool(votes_array, ksize=(1, cell_size, cell_size, bins), strides=(1, cell_size, cell_size, 1))
    
    return cell_array
    # Batch Level
    

In [15]:
sess = tf.InteractiveSession()
x = tf.convert_to_tensor(fake_batch)

In [16]:
output = hog_layer(x)

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [ ]:
def hog_layer_outputshape(input_shape):
    #shape = list(input_shape)
    #assert len(shape) == 2  # only valid for 2D tensors
    #shape[-1] *= 2
    return tuple(shape)

In [ ]:
def block_operations(cell_array):
    

In [ ]:
# Model 
inputs = Input(shape(89, 89))
x_conv = conv2d(1, (3,3), strides=1, padding="same", kernel_initializer=init_prewitt_x)(inputs)
y_conv = conv2d(1, (3,3), strides=1, padding="same", kernel_initializer=init_prewitt_y)(inputs)
conv_stacked = keras.layers.Concatenate(axis=-1)([x_conv, y_conv])
x = Lambda(hog_layer, output_shape=hoglayer_output_shape)(conv_stacked)

In [ ]:
# add a layer that returns the concatenation
# of the positive part of the input and
# the opposite of the negative part

def antirectifier(x):
    # Splitting the concatenated later into it's sperate components
    axis = len(tf.shape(x)) - 1
    
    return K.concatenate([pos, neg], axis=1)

def antirectifier_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 2  # only valid for 2D tensors
    shape[-1] *= 2
    return tuple(shape)

model.add(Lambda(antirectifier,
                 output_shape=antirectifier_output_shape))

In [5]:
sess = tf.InteractiveSession()
x = tf.convert_to_tensor(prewitt_x)

In [21]:
print (x.eval())
ind = tf.where(x < 0)
ind

[[-1  0  1]
 [-1  0  1]
 [-1  0  1]]


<tf.Tensor 'Where_3:0' shape=(?, 2) dtype=int64>

In [22]:
x[ind]

ValueError: Shape must be rank 1 but is rank 3 for 'strided_slice' (op: 'StridedSlice') with input shapes: [3,3], [1,?,2], [1,?,2], [1].

In [24]:
bins = 8         # number of gaps
w = 360/bins     # gap size
centers = np.arange(0, 360, w)+0.5*w

In [25]:
centers

array([  22.5,   67.5,  112.5,  157.5,  202.5,  247.5,  292.5,  337.5])

In [15]:
def calc_bin(angle):
    return np.floor((angle/w)-0.5)%bins

In [16]:
calc_bin(22.4)

7.0